In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from google.colab import files
from google.colab import drive
import numpy as np
#@title Import Statements
!pip install PyGithub

# Package Imports
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from urllib.request import urlopen # The default requests package
import requests # For making GitHub requests
from pprint import pprint # For pretty printing
from pathlib import Path # The Path class

# For the more advanced requests
import base64
import os
import sys
sys.path.append("./PyGithub");
from github import Github
from getpass import getpass


     |████████████████████████████████| 296kB 22.7MB/s 


In [3]:
#@title Function Definitions
# Recursively Import the Data (AUTOMATIC)

def _decode_and_write(file__, path_):
    data = file__.decoded_content
    data = data.decode('utf-8')[1:]
    with open(path_, 'w') as writefile:
        writefile.write(data) 
    data = data.splitlines()
    data_rows = []
    for count, word in enumerate(data):
        if count>0:
            data_rows.append(word.split(','))
    data = pd.DataFrame(data_rows)
    data = data.to_numpy()
    return data


def import_github(path_name="AllLyrics.txt"):
    """
    Function for importing the github file
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()
    print("Importing Github cleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    RAP_DATA = []
    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the Lyrics
            RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
    
    temp_path = Path(path_name)
    if temp_path.is_file(): 
        if os.stat(path_name).st_size == 0:
            write_bool2 = True
        else: 
            write_bool2 = False
    else: 
        write_bool2 = True
    
    if write_bool2: 
        for lyric in RAP_DATA: 
            try:
                with open(path_name, 'w') as writefile: 
                    writefile.write(lyric)
            except: 
                print("Error, file moved/deleted during write")
        print("{} is now up to date!".format(path_name))
    else: 
        print("{} is already up to date!".format(path_name))
    
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 

        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        path = path[title_start + 6:title_start + title_len + 4]

        print("Writing file {} {}".format(counter, path))
        temp_path = Path(path)
        if temp_path.is_file():
            with open(path,'w'): pass # Cheeky way to clear the file if it exists
        
        # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy array 
        data = file_.decoded_content
        data = data.decode('utf-8')[1:]

        with open(path, 'w') as writefile:
            writefile.write(data) 
        print("All files now up to date!")


def update_github(write_bool=False, path_name="AllLyrics.txt"):
    """
    Function for updating the github file, by cleaning the lyrics, optional write to txt file. 
    write_bool: bool
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()

    print("Importing editing csv files...")

    # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path 
        path = str(path)
        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len + 4]
        print("Writing file {} {}".format(counter, name))
        if name.lower() == "censors.csv":
            censors = _decode_and_write(file_, path)
        elif name.lower() == "capitals.csv":
            capitals = _decode_and_write(file_, path)
        else: 
            _decode_and_write(file_, path)
    print("All editing csv files are up to date!")

    print("Importing Github uncleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/UNCLEAN", ref=branch)

    RAP_DATA = []
    rap_lyric_names = []

    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('UNCLEAN')
            title_len = path[title_start:].find('.')
            name = path[title_start + 8:title_start + title_len]
            if name[-2:] == 'UC':
                name = name[:-2]
            rap_lyric_names.append(name) 

        # Append the Lyrics
        RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
        
    # Remove the \ufeff at the beginning O(n)
    for count, lyric in enumerate(RAP_DATA): 
        RAP_DATA[count] = lyric[1:]

    # Censor the profanities O(n*m + n*m2) m > m2 xor m2 > m
    for count in range(len(RAP_DATA)): 
        for i in range(len(censors[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))
        for i in range(len(capitals[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(capitals[i, 0]), str(capitals[i, 1]))

    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    cleaned_names = []
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 
        print("File {} ".format(counter + 1) + path)
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('CLEAN')
            title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len]
        if name[-2:] == 'CL':
            name = name[:-2]
        cleaned_names.append(name) 

    # ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH' to avoid overwriting import changes
    # If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
    # If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
    # print(rap_lyric_names)
    # print(cleaned_names)
    print("Commiting files to github...")
    for counter, new_name in enumerate(rap_lyric_names): 
        if new_name in cleaned_names: 
            duplicate = r_proj_clone.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref=branch)
            r_proj_clone.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch=branch)
        else:
            r_proj_clone.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch=branch)

    if write_bool: 
        print("Writing text file to: {}".format(path_name))
        with open(path_name, 'w') as writefile:
            for lyric in RAP_DATA:
                writefile.write(lyric)

In [4]:
# Import all of Mike's lyrics. PATKEY: 5ae2446bd5828c9e27deb3865118d9e783aa6e15
import_github()
Text = open("AllLyrics.txt", "r").read()
# turn text to lower case to reduce vocabulary
Text = Text.lower()
with open("AllLyrics.txt", "r") as f:
    content = f.readlines()
# Defined content in case we need to split each line/bar into strings
content = [x.strip() for x in content] 
print(content)
Vocabulary = ''.join([i for i in Text if not i.isdigit()]).replace("\n"," ").split(' ')
print(Vocabulary)

Enter your PAT key ··········
Main Branch: Yes or No? Yes
Importing Github cleaned text files...
AllLyrics.txt is now up to date!
Writing file 0 capitals.csv
All files now up to date!
Writing file 1 censors.csv
All files now up to date!
['baby It was all a dream', 'I used to read Word Up magazine', 'Salt n Pepa and Heavy D up in the limousine', 'Hangin pictures on my wall', 'Every Saturday Rap Attack Mr Magic Marley Marl', 'I let my tape rock til my tape popped', 'Smokin sandwhiches and Bambu sippin on Private Stock', 'Way back when I had the red and black lumberjack', 'With the hat to match', 'Remember Rappin Duke duhha duhha', 'You never thought that hip hop would take it this far', 'Now Im in the limelight cause I rhyme tight', 'Time to get paid blow up like the World Trade', 'Born sinner the opposite of a winner', 'Remember when I used to eat sardines for dinner', 'Peace to Ron G Brucey B Kid Capri', 'Funkmaster Flex Lovebug Starski wassup', 'Im blowin up like you thought I would',

In [5]:
# List of all unique vocabulary in alphabetical order
words = sorted(list(set(Vocabulary)))
print(words)
vocab_size = len(words)
# Need to create a dictionary as LTSM model needs numbers, not words (will word this better)
intdict = { i : words[i] for i in range(len(words))}
print(intdict)
# Need to reverse this at the end to reverse numbers back into words
charsdict = { words[i] : i for i in range(len(words))}
print(charsdict)

['', '#', 'a', 'aaalll', 'aand', 'able', 'abortion', 'about', 'above', 'abuser', 'abusers', 'ac', 'accept', 'acclaimed', 'account', 'accountant', 'ace', 'across', 'acs', 'act', 'actin', 'acting', 'action', 'actors', 'acts', 'actually', 'adapt', 'add', 'address', 'admit', 'adolescents', 'advance', 'advanced', 'advancin', 'advancing', 'adventure', 'aes', 'affect', 'afraid', 'africans', 'after', 'again', 'against', 'age', 'agenda', 'agents', 'agreed', 'ahh', 'aids', 'aight', 'aim', 'aimin', 'aint', 'air', 'airport', 'aiy', 'ak', 'aka', 'akon', 'alarm', 'alberta', 'album', 'alcoholic', 'ale', 'alfred', 'alibi', 'alive', 'alize', 'all', 'alligators', 'allow', 'allowed', 'almost', 'alone', 'along', 'alot', 'already', 'alright', 'also', 'always', 'am', 'amateurs', 'americas', 'amnesia', 'amount', 'amped', 'amtrak', 'amuck', 'an', 'and', 'ands', 'angel', 'animal', 'animals', 'ankles', 'annihilate', 'announced', 'anoint', 'anotha', 'another', 'answer', 'antartic', 'anthem', 'any', 'anybody', 'a

In [6]:
def word_count(lyrics):
  a = {}
  for word in Vocabulary:
    if word in a:
      a[word] += 1
    else:
      a[word] = 1
  return a
word_dict = word_count(Vocabulary)
sort_dict = sorted(word_dict.items(), key = lambda x: x[1], reverse = True)
# Top 20 words
sort_dict1 = sort_dict[:20]
sort_dict1

[('the', 2126),
 ('i', 1421),
 ('you', 1323),
 ('to', 1072),
 ('and', 904),
 ('my', 699),
 ('a', 692),
 ('in', 580),
 ('me', 562),
 ('on', 500),
 ('your', 494),
 ('that', 431),
 ('it', 423),
 ('im', 394),
 ('what', 388),
 ('with', 368),
 ('is', 353),
 ('like', 350),
 ('for', 330),
 ('ninja', 330)]

In [7]:
# seq_len depicts how many words are in our training set per training example
seq_len = 20 
sentences = [] 
# nextword will contain all the associated 'next words' in our sentences
# Ex. for the sentence 'I am doing a NLP project' with seq_len = 5, sentences = [I am doing a NLP] and nextword = [project]
nextword = [] 

# We need training and test examples so we split our text into chunks of 'seq_len' with the next word in the list being our 'test data'
for i in range(0, len(Vocabulary) - seq_len):
    train_set  = Vocabulary[i:i+seq_len]
    # Uncomment next line to check it is working
    #print(train_set)
    # Output sequence (will be used as target)
    test_set = Vocabulary[i + seq_len]
    #print(test_set)
    # We append and convert words to digits
    sentences.append([charsdict[word] for word in train_set])
    # Store targets in data_y`
    nextword.append(charsdict[test_set])
#sentences = list(sentences)
#nextword = list(nextword)
print(sentences[46215])
print(nextword[46215])
size_training = len(sentences)
print('The number of sentences in our training set', size_training)

[2105, 1617, 2884, 4852, 4742, 2186, 4464, 1220, 4559, 2356, 4464, 550, 2521, 2, 2689, 2334, 5057, 1929, 2025, 4886]
0
The number of sentences in our training set 46216


In [8]:
from keras.utils import to_categorical
y_train = to_categorical(nextword) # num_classes = (len(words)))
# Need to reshape so in right format for LSTM
x_train = np.reshape(sentences,(size_training, seq_len, 1))
print(x_train.shape)
print(y_train.shape)
# Normalising X - Took too much RAM
x_train = x_train / float(vocab_size)
x_train.shape
x_train[1]

(46216, 20, 1)
(46216, 5069)


array([[4.43085421e-01],
       [9.54626159e-01],
       [1.34148747e-02],
       [3.94555139e-04],
       [2.46794239e-01],
       [4.25724995e-01],
       [9.37463010e-01],
       [8.99388440e-01],
       [6.85934109e-01],
       [9.84217794e-01],
       [9.35490235e-01],
       [5.20023673e-01],
       [7.29335175e-01],
       [5.69737621e-01],
       [6.24975340e-01],
       [1.75577037e-02],
       [3.91004143e-01],
       [2.03985007e-01],
       [9.35490235e-01],
       [4.31248767e-01]])

In [9]:
# Need to compile model now
from keras.layers import Dropout
from keras.optimizers import SGD

model = Sequential()
model.add(LSTM(256, return_sequences = True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01))




In [10]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size = 200, epochs = 500)

Epoch 1/500
232/232 [==============================] - 12s 12ms/step - loss: 6.8644 - accuracy: 0.0406
Epoch 2/500
232/232 [==============================] - 3s 12ms/step - loss: 6.5269 - accuracy: 0.0455
Epoch 3/500
232/232 [==============================] - 3s 12ms/step - loss: 6.4874 - accuracy: 0.0464
Epoch 4/500
232/232 [==============================] - 3s 12ms/step - loss: 6.4714 - accuracy: 0.0470
Epoch 5/500
232/232 [==============================] - 3s 12ms/step - loss: 6.5042 - accuracy: 0.0441
Epoch 6/500
232/232 [==============================] - 3s 12ms/step - loss: 6.4730 - accuracy: 0.0469
Epoch 7/500
232/232 [==============================] - 3s 12ms/step - loss: 6.4732 - accuracy: 0.0475
Epoch 8/500
232/232 [==============================] - 3s 12ms/step - loss: 6.4755 - accuracy: 0.0440
Epoch 9/500
232/232 [==============================] - 3s 12ms/step - loss: 6.5010 - accuracy: 0.0434
Epoch 10/500
232/232 [==============================] - 3s 12ms/step - loss: 6.55